In [1]:
import boto3
import csv
from cryptography.fernet import Fernet
import json
import base64

In [2]:
iam = boto3.client('iam')
response = response = iam.get_user()
user_name = response['User']['UserName']
user_name

'rspamzn'

In [ ]:
iam = boto3.client("iam")

policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowCreateKey",
            "Effect": "Allow",
            "Action": "kms:*",
            "Resource": "*"
        }
    ]
}

response = iam.create_policy(
    PolicyName = 'AllowCreateKey',
    PolicyDocument = json.dumps(policy_document)
)

policy_arn = response['Policy']['Arn']

In [ ]:
iam.attach_user_policy(
    UserName = user_name,
    PolicyArn = policy_arn
)

In [3]:
kms = boto3.client('kms')

def create_key():
    response = kms.create_key(
        Description='Key for PII encryption',
        KeyUsage='ENCRYPT_DECRYPT',
        CustomerMasterKeySpec='SYMMETRIC_DEFAULT',
        Origin='AWS_KMS'
    )
    return response['KeyMetadata']['KeyId']

In [4]:
key_id = create_key()

In [5]:
key_id

'8b6c25ff-be39-4b29-ac55-83c7cd8a9540'

In [8]:
def retrieve_key(key_id):
    response = kms.describe_key(KeyId=key_id)
    return response['KeyMetadata']['Arn']

key = retrieve_key(key_id)

response = kms.generate_data_key(KeyId=key, KeySpec='AES_256')
plaintext_datakey = response['Plaintext']
encrypted_datakey = response['CiphertextBlob']
print(plaintext_datakey)
print(encrypted_datakey)
fernet_key = base64.urlsafe_b64encode(plaintext_datakey)
fernet_key

b'\xdauX\x8b\xc2.\x11+s\x81\x95\xf0\xa8\x1a\x13\xad\x89\xbf\xa9\x915x\xd5\xbau\x89\x9b.F[\x83\x7f'
b'\x01\x02\x03\x00x\x89\xe52V\xf8NR\x99\xc3\x80Py\x9e\x91\xd5\xa2\xc2\x1f\x85\x08}t\xc8o4\xcf\xd0\x00\xe3\xd1\xcf"\x014\x95An\xfc\xc9\x12\x1e\xb6\xbe\xa1\xbf\xbekH\xd9\x00\x00\x00~0|\x06\t*\x86H\x86\xf7\r\x01\x07\x06\xa0o0m\x02\x01\x000h\x06\t*\x86H\x86\xf7\r\x01\x07\x010\x1e\x06\t`\x86H\x01e\x03\x04\x01.0\x11\x04\x0c\xad\x01\x9bQ\xb1x\x112\x0b\xf8\x01\xe6\x02\x01\x10\x80;\xed\xb7i\xa0\xb2}L\x8b\xbd\xd36b\x98I\xce\xe8\xb25!\x8e\xee\xf1Z\x0b\xcc\x1f>\x1f\x17\x8a\xbe^\x06H\xaeM\x8c\xae\xd4\xb9xn\x0e\x9f\xc2b?87n$\xa8\xb7\xd3Y\'\xc5\x88\xcd'


b'2nVYi8IuEStzgZXwqBoTrYm_qZE1eNW6dYmbLkZbg38='

In [54]:
cipher_suite = Fernet(fernet_key)
cipher_suite

In [55]:
cipher_text = cipher_suite.encrypt("Hello Snickers. Come to me!".encode('utf-8'))
cipher_text


b'gAAAAABnATmiCVHireMgYwx6bgunAfkyuOhz8p2YaJt3an7V_UJhaQhubN0hztYY_lGf2z6ldRFWVykqIezKRo2rXKah_6uf4auJT4j3Y6ETgI-CjRkTj_0='

In [56]:
plain_text = cipher_suite.decrypt(cipher_text).decode('utf-8')
plain_text

'Hello Snickers. Come to me!'

In [9]:
print(kms.list_keys())

{'Keys': [{'KeyId': '1a8e1595-b5a8-45c3-ab28-bacbd69b627f', 'KeyArn': 'arn:aws:kms:ap-southeast-1:590183920655:key/1a8e1595-b5a8-45c3-ab28-bacbd69b627f'}, {'KeyId': '8b6c25ff-be39-4b29-ac55-83c7cd8a9540', 'KeyArn': 'arn:aws:kms:ap-southeast-1:590183920655:key/8b6c25ff-be39-4b29-ac55-83c7cd8a9540'}, {'KeyId': 'bf8cce1e-fb5f-42f8-8038-c7717880955e', 'KeyArn': 'arn:aws:kms:ap-southeast-1:590183920655:key/bf8cce1e-fb5f-42f8-8038-c7717880955e'}], 'Truncated': False, 'ResponseMetadata': {'RequestId': '897e8bc2-8b69-4070-813e-abc540b909ae', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '897e8bc2-8b69-4070-813e-abc540b909ae', 'cache-control': 'no-cache, no-store, must-revalidate, private', 'expires': '0', 'pragma': 'no-cache', 'date': 'Sun, 06 Oct 2024 03:06:20 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '464', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [32]:
from Crypto.Cipher import AES
from Crypto.Util import Counter
import os

key = os.urandom(16)  # AES key must be either 16, 24, or 32 bytes long
iv = os.urandom(8)   # Initialization vector (IV)

# Create a counter object
ctr = Counter.new(64, prefix=iv)

cipher = AES.new(key, AES.MODE_CTR, counter=ctr)

plaintext = b'Hello, Snickers, come here!'
ciphertext = cipher.encrypt(plaintext)
base64_encoded = base64.b64encode(ciphertext).decode('utf-8')
print(ciphertext)
print(base64_encoded)
print(f"len of plaintext: {len(plaintext)}")
print(f"len of cyphertext: {len(ciphertext)}")
print(f"len of base64encoded: {len(base64_encoded)}")

# Decrypting
cipher_decrypt = AES.new(key, AES.MODE_CTR, counter=ctr)
decrypted = cipher_decrypt.decrypt(base64.b64decode(base64_encoded)).decode('utf-8')

print(decrypted)  # Outputs: b'Hello, World!'

b':\xb1\x85\x1bT\xe5[\xd4\xa8n\xbeA\xc4Z\xf5\\\xf8\xd2\xc4Bs\x1b\xc3\xfb\x06\xf1\xc1'
OrGFG1TlW9Sobr5BxFr1XPjSxEJzG8P7BvHB
len of plaintext: 27
len of cyphertext: 27
len of base64encoded: 36
Hello, Snickers, come here!
